In [1]:
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
DEV = True

if DEV:
    model_name = "avsolatorio/GIST-small-Embedding-v0"
else:
    model_name = "avsolatorio/GIST-Embedding-v0"


# model_name = 'jinaai/jina-embeddings-v2-small-en'

In [3]:
def import_labelled_data(path="data/level-0.5/data.json"):
    data = pd.read_json(path, encoding="latin-1")
    return data


data = import_labelled_data(path="../../../data/level-0.5/data.json", )



# train test split
from sklearn.model_selection import train_test_split

# 0.49, 0.21, 0.3 split
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.3, random_state=42)

display(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19368 entries, 0 to 19367
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           19368 non-null  object
 1   text          19368 non-null  object
 2   relevance     19368 non-null  object
 3   multiclasses  19368 non-null  object
dtypes: object(4)
memory usage: 605.4+ KB


None

,url,text,relevance,multiclasses
0,https://www.conservationevidence.com/synopsis/...,1 \n \n \n2 \n \n \nSubtidal Benthic Invertebr...,relevant,[Marine Invertebrates]
1,https://www.conservationevidence.com/synopsis/...,\n \n \n Control of freshwater \n invasi...,relevant,"[Rivers and Lakes, Invasive, Fish]"
2,https://www.conservationevidence.com/synopsis/...,1 \n \nGrassland Conservation \n2 \n \nGrassla...,relevant,[Grassland]
3,https://www.conservationevidence.com/synopsis/...,\n \n \nii \n \n \n \n \n \n \n \nPrimate Co...,relevant,[Mammals]
4,https://www.conservationevidence.com/synopsis/...,CONSERVATION EVIDENCE SERIES SYNOPSES\nTerrest...,relevant,[Mammals]


In [4]:
from chunking import chunk_dataset_and_explode


# roughly 4 characters per token
max_len = 512

train_data = chunk_dataset_and_explode(train_data, max_len=max_len, overlap=int(max_len * 0.2))
test_data = chunk_dataset_and_explode(test_data, max_len=max_len, overlap=int(max_len * 0.2))
val_data = chunk_dataset_and_explode(val_data, max_len=max_len, overlap=int(max_len * 0.2))

  0%|          | 0/9489 [00:00<?, ?it/s]

  0%|          | 0/5811 [00:00<?, ?it/s]

  0%|          | 0/4068 [00:00<?, ?it/s]

In [5]:
def stratified_sample(dataset,label_column: str = 'relevance',num_samples_per_label: int = 100):
    return (
        dataset
        .sample(frac=1,random_state=42)
        .groupby(label_column)[dataset.columns]
        .apply(lambda x: x.sample(min(num_samples_per_label,len(x)),random_state=42),include_groups=True).reset_index(drop=True)
    )


if DEV:
    train_data = stratified_sample(train_data,num_samples_per_label=1000)
    val_data = val_data.sample(100,random_state=42)
    test_data = test_data.sample(200,random_state=42)
else:
    train_data = train_data.sample(frac=1,random_state=42)
    val_data = val_data.sample(500,random_state=42)
    test_data = test_data.sample(frac=1,random_state=42)


In [6]:
train_data

,chunk_id,url,text,relevance,multiclasses
0,17236,http://www.world-psi.org/sites/default/files/d...,"struction, n u Marchands m s e c i Source: WIE...",irrelevant,[]
1,17698,https://deerruncdd.com/wp-content/uploads/2024...,"ctly prohibited at the pool, with the exceptio...",irrelevant,[]
2,9087,https://www.mwe.go.ug/sites/default/files/libr...,ined after infection has occurred. These objec...,irrelevant,[]
3,13668,https://documents1.worldbank.org/curated/en/57...,on and new sources of FX income. 233. This are...,irrelevant,[]
4,11745,https://worldjusticeproject.org/sites/default/...,Leona África Subsahariana Bajo Serbia Eastern...,irrelevant,[]
...,...,...,...,...,...
1995,78,https://www.wetlands.org/wp-content/uploads/20...,"pecialists in each flamingo range state, those...",relevant,[Animals Ex-Situ]
1996,4747,https://www.conservationevidence.com/individua...,"ge, compared to conventional tillage, in two o...",relevant,[Farmland]
1997,4030,https://www.conservationevidence.com/individua...,ubterranean clover Trifolium subterraneum and ...,relevant,[Farmland]
1998,13,https://www.conservationevidence.com/synopsis/...,"ackground \nIn captivity, amphibian diets are ...",relevant,"[Animals Ex-Situ, Captivity]"


In [7]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data, split="train")
test_dataset = Dataset.from_pandas(test_data, split="test")
val_dataset = Dataset.from_pandas(val_data, split="val")

train_dataset

Dataset({
    features: ['chunk_id', 'url', 'text', 'relevance', 'multiclasses'],
    num_rows: 2000
})

In [8]:

#! had to modify FastFitTrainer to at /fastfit/train.py, line 879, to add trust_remote_code=True to the loading of 'accuracy' metrics
#! don't know why it's not default, since accuracy is the default in fastfit



#! IMPORTANT: another change in FastFitTrainer, also at line 879; comment out and replace the fixed version above
#! since load_metric is deprecated in favour of evaluate.load()
#! added functionality for sending in multiple metrics to evaluate at once
#! added macro averages for non-accuracy metrics too
#! essentially, copy the below code to replace line 879
#! and delete the compute_metrics function already there

### Insert into line 879.

```python
        # metric = load_metric(self.data_args.metric_name, experiment_id=uuid.uuid4())
        from evaluate import combine, load
        if type(self.data_args.metric_name) == str: # single metric name
            metrics = [load(self.data_args.metric_name, experiment_id=uuid.uuid4())]
        elif type(self.data_args.metric_name) == list: # compute multiple metrics
            metrics = [load(metric,experiment_id=uuid.uuid4()) for metric in self.data_args.metric_name]

        # You can define your custom compute_metrics function. It takes an `EvalPrediction` object (a namedtuple with a
        # predictions and label_ids field) and has to return a dictionary string to float.
        def compute_metrics(p: EvalPrediction):
            predictions = (
                p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
            )
            predictions = (
                np.squeeze(predictions)
                if self.is_regression
                else np.argmax(predictions, axis=1)
            )
            references = p.label_ids

            results = {}

            for metric in metrics:
                if metric.name != 'accuracy':
                    results.update(metric.compute(predictions=predictions, references=references,average='macro'))
                else:
                    results.update(metric.compute(predictions=predictions, references=references))

            return results
    ```

In [9]:
from fastfit import FastFitTrainer

# same args as the huggingface TrainingArguments
if DEV:
    output_dir = f'models/relevance/dev/{model_name}'
else:
    output_dir = f'models/relevance/{model_name}'

trainer = FastFitTrainer(
    model_name_or_path=model_name,
    train_dataset=train_dataset,
    validation_dataset=val_dataset,
    test_dataset=test_dataset,
    output_dir=output_dir,
    overwrite_output_dir=True,
    label_column_name='relevance',
    text_column_name="text",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    max_text_length=max_len,
    num_repeats=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy='epoch',
    metric_name=['precision','recall','f1','accuracy'],
    load_best_model_at_end=True,
    metric_for_best_model='precision',
    fp16=True,
    fp16_opt_level="O2",
    fp16_full_eval=True,
)

08/16/2024 09:52:14 - WARNING - fastfit.train - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: True


/workspace/GreyLit/venv/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/2000 [00:00<?…

Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/100 [00:00<?,…

Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/200 [00:00<?,…

Running tokenizer on dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

In [10]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

print(torch.version.cuda)

torch.cuda.empty_cache()
print(trainer.model.device)

True
0
NVIDIA GeForce RTX 3090
12.1
cuda:0


In [11]:

#! another fastfit library modification
#! in /fastfit/train.py, line 971, cast ignore_keys_for_eval from type set to a list
#! since it gets concatenated to a list later on, and otherwise crashes
#! note that since we've added lines above, this is now line 1003
#! the line beginning ignore_keys_for_eval={"doc_input_ids","doc_attention_mask","labels"}


model = trainer.train()

[WARNING|modeling_utils.py:1198] 2024-08-16 09:52:34,372 >> Could not estimate the number of tokens of the input, floating-point operations will not be computed


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,3.741300,3.926953,0.833333,0.978261,0.888889,0.960000


In [ ]:
results = trainer.evaluate()

***** eval metrics *****
  epoch                   =        5.0
  eval_accuracy           =       0.97
  eval_f1                 =     0.8919
  eval_loss               =     1.8722
  eval_precision          =     0.8333
  eval_recall             =      0.984
  eval_runtime            = 0:00:01.52
  eval_samples            =        100
  eval_samples_per_second =     65.462
  eval_steps_per_second   =     16.365


In [ ]:
print(f'Accuracy: {results["eval_accuracy"]}')

Accuracy: 0.97


In [ ]:
results = trainer.test()

***** test metrics *****
  epoch                   =        5.0
  eval_accuracy           =       0.97
  eval_f1                 =     0.9167
  eval_loss               =     1.8095
  eval_precision          =     0.8696
  eval_recall             =     0.9836
  eval_runtime            = 0:00:02.96
  eval_samples_per_second =     67.509
  eval_steps_per_second   =     16.877
  test_samples            =        200


In [ ]:
print(f'Estimate of files processed per second:', results["eval_samples_per_second"]*len(train_data['chunk_id'].unique())/len(train_data))

Estimate of files processed per second: 36.792405


In [ ]:

# if DEV:
#     from os import makedirs, path
#     if not path.exists(f'models/relevance/dev/{model_name}'):
#         makedirs(f'models/relevance/dev/{model_name}')

#     model.save_pretrained(f'models/relevance/dev/{model_name}')

# else:
#     model.save_pretrained(f'models/relevance/{model_name}')